In [ ]:
from flask import Flask, request, jsonify
import torch
from torchvision import transforms
from PIL import Image
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from main import Item, sqlite_engine  # Import Item model and database connection from main.py




In [ ]:
# Load trained model
model = torch.load("ML_Model.pth", map_location=torch.device("cpu"))
model.eval()

# Define image preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
])

# Create Flask app
app = Flask(__name__)

# Create a session for database operations
SQLiteSession = sessionmaker(bind=sqlite_engine)
sqlite_session = SQLiteSession()

In [ ]:
@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return jsonify({"error": "No image uploaded"}), 400

    file = request.files['image']
    img = Image.open(file)
    img = transform(img).unsqueeze(0)

    with torch.no_grad():
        output = model(img)
        predicted_class = torch.argmax(output, dim=1).item()

    # Map predicted class index to item name
    class_mapping = {0: "resistor", 1: "LED", 2: "capacitor"}  # Update with actual classes
    item_name = class_mapping.get(predicted_class, "unknown")

    # Update item quantity in the SQLite database
    item = sqlite_session.query(Item).filter_by(name=item_name).first()
    if item:
        item.quantity -= 1  # Reduce stock by 1
        sqlite_session.commit()
        return jsonify({"predicted_class": item_name, "updated_quantity": item.quantity})
    else:
        return jsonify({"error": "Item not found in database"}), 404

if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)  # Run Flask server